In [1]:
# Импорт библиотек
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Input, Flatten, Dense, Dropout)
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

# Параметры
img_size = (64, 64)
batch_size = 32

# Нормализация
data = ImageDataGenerator(rescale=1./255)

# Данные
train_data = data.flow_from_directory(
    'Train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

test_data = data.flow_from_directory(
    'Test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 6589 images belonging to 4 classes.
Found 2313 images belonging to 4 classes.


In [2]:
from tensorflow.keras.applications import ResNet50

# Предобучение
base_model = ResNet50(weights='imagenet', include_top=False, 
                      input_shape=(64, 64, 3))
base_model.trainable = False
    
# Архитектура модели
model_resnet = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

# Компиляция
model_resnet.compile(optimizer='adam', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

#model.summary()

In [3]:
from tensorflow.keras.applications import EfficientNetB3

base_model = EfficientNetB3(weights='imagenet', include_top=False, 
                            input_shape=(64, 64, 3))
base_model.trainable = False

model_effnet = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model_effnet.compile(optimizer='adam', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

In [4]:
from tensorflow.keras.applications import MobileNetV3Small

base_model = MobileNetV3Small(weights='imagenet', include_top=False, 
                              input_shape=(64, 64, 3))
base_model.trainable = False

model_mobile = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model_mobile.compile(optimizer='adam', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

In [5]:
# Обучение
epochs = 3

history_resnet = model_resnet.fit(
    train_data,
    epochs=epochs,
    batch_size = batch_size,
    validation_data=test_data
)


Epoch 1/3
206/206 ━━━━━━━━━━━━━━━━━━━━ 137s 619ms/step - accuracy: 0.5103 - loss: 0.9262 - val_accuracy: 0.6459 - val_loss: 0.6922
Epoch 2/3
206/206 ━━━━━━━━━━━━━━━━━━━━ 60s 291ms/step - accuracy: 0.6122 - loss: 0.6915 - val_accuracy: 0.6576 - val_loss: 0.6457
Epoch 3/3
206/206 ━━━━━━━━━━━━━━━━━━━━ 58s 283ms/step - accuracy: 0.6447 - loss: 0.6666 - val_accuracy: 0.6904 - val_loss: 0.6089


In [6]:
history_effnet = model_effnet.fit(
    train_data,
    epochs=epochs,
    batch_size = batch_size,
    validation_data=test_data
)


Epoch 1/3
206/206 ━━━━━━━━━━━━━━━━━━━━ 81s 280ms/step - accuracy: 0.4448 - loss: 1.3642 - val_accuracy: 0.4272 - val_loss: 1.2830
Epoch 2/3
206/206 ━━━━━━━━━━━━━━━━━━━━ 56s 268ms/step - accuracy: 0.4833 - loss: 1.1229 - val_accuracy: 0.4272 - val_loss: 1.2097
Epoch 3/3
206/206 ━━━━━━━━━━━━━━━━━━━━ 41s 200ms/step - accuracy: 0.4969 - loss: 1.1338 - val_accuracy: 0.4272 - val_loss: 1.1991


In [ ]:
history_mobile = model_mobile.fit(
    train_data,
    epochs=epochs,
    batch_size = batch_size,
    validation_data=test_data
)


Epoch 1/3


In [ ]:

# Оценка моделей
def show(model, history):
    # Предсказания
    y_pred = model.predict(test_data)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = test_data.classes
    
    # Отчёт
    print(classification_report(y_true_classes, 
                                y_pred_classes, 
                                target_names=list(test_data.class_indices.keys())))
    
    # Матрица ошибок
    matrix = confusion_matrix(y_true_classes, y_pred_classes)
    
    plt.figure(figsize=(8,6))
    sns.heatmap(matrix, 
                annot=True, 
                fmt='d', 
                cmap='Blues', 
                xticklabels=test_data.class_indices.keys(), 
                yticklabels=test_data.class_indices.keys())
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
    
    # Графики точности и потерь
    plt.figure(figsize=(14,5))
    
    # Accuracy
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Изменение точности модели')
    
    # Loss
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Изменение функции потерь')
    
    plt.show()

show(model_resnet, history_resnet)

In [ ]:
show(model_effnet, history_effnet)

In [ ]:
show(model_mobile, history_mobile)